In [1]:
import datetime
import json
import os
import random
import sys
import warnings
import pandas as pd

sys.path.append(os.path.join(os.getcwd(), '../../../common'))
from ppg_feature_processor import generate_train_feature

# Suppress specific warnings
warnings.filterwarnings("ignore", category=DeprecationWarning, message=".*Conversion of an array with ndim > 0 to a scalar is deprecated.*")

In [2]:
# 读取 Excel 文件'
middleSchool_userProfile_file_new = './outdir/MiddleSchool_Depression_Anxiety.xlsx'
middleSchool_userProfile_file = r'.././outdir/MiddleSchool_Depression_Anxiety.xlsx'
hospital_userProfile_file = '../../../../Data/unhealthy_clinic_results.xlsx'
university_userProfile_file = '../../../../Data/healthy_scale_results.xlsx'  # 将此替换为您的 Excel 文件路径

In [3]:
depression_positive_threshold = 3
depression_negative_threshold = 0

In [22]:
### READ THE MIDDLESCHOOL SUBJECTS old
df_middle_old = pd.read_excel(middleSchool_userProfile_file)
df_middle_old_pos = df_middle_old[df_middle_old['depression_label'] == 1]
df_middle_old_neg = df_middle_old[df_middle_old['depression_label'] == 0]

middle_subjects_old_pos = df_middle_old_pos['user_id'].values.tolist()
middle_subjects_old_neg = df_middle_old_neg['user_id'].values.tolist()
print(f"middle depression positive subjects count old: {len(middle_subjects_old_pos)}")
print(f"middle depression positive subjects count old: {len(middle_subjects_old_neg)}")

middle depression positive subjects count old: 62
middle depression positive subjects count old: 228


In [23]:
### READ THE MIDDLESCHOOL SUBJECTS
df_middle = pd.read_excel(middleSchool_userProfile_file_new, sheet_name= 'INMHT')
middle_subjects = df_middle['subjectid'].values.tolist()
print(f"middle depression positive subjects count: {len(middle_subjects)}")

middle depression positive subjects count: 128


In [20]:
profile_dict = dict()

for _, row in df_middle.iterrows():
    userId = row['subjectid']
    gender = row['gender']
    age = row['age']
    profile_dict[userId] = (gender, age)

for _,row in df_middle_old.iterrows():
    userId = row['user_id']
    gender = row['gender']
    age = row['age']
    profile_dict[userId] = (gender, age)



In [21]:
def load_wave_middleSchool(path):
    try:
        with open(path, 'r', encoding='utf-8') as file:  
            data = json.load(file)
        return data["ppg_raw_data"]
    except:
        print("Subject Not Loaded")
        pass

def load_wave_middleSchool_old(path):
    try:
        with open(path, 'r', encoding='utf-8') as file:  
            data = json.load(file)
            list_data = data['RawResults'][0]['rawDatas']
            final = [i/100000 for i in list_data]
        return final
    except (KeyError, IndexError, TypeError):
        with open(path, 'r', encoding='utf-8') as file:
            data = json.load(file)
        return data["datas"]
    

In [28]:
sample_freq = 125
sample_window_minute = 1.0

middle_file_old_dir = r'C:\Users\Administrator\Desktop\PPG_dataset\PPG_Dataset_ALL\middle_school\ppg_data'
middle_file_dir = r'C:\Users\Administrator\Desktop\middle_school_data\ppg_data'
hospital_file_dir = r'C:\Users\Administrator\Desktop\PPG_dataset\PPG_Dataset_ALL\unhealthy'

In [29]:
all_subjects_neg = middle_subjects_old_neg + middle_subjects

In [30]:
def generate_dataset(subject_collection, label, data_aug_times=1):
    ignored_id = []
    dataset = pd.DataFrame()
    for subject in subject_collection:
            subject_wave = []

            if subject in (middle_subjects) and subject not in ['ee2ffda0-b7df-d854-7b31-d2e714196949', '13cdee35-1e76-9aed-8c90-9c5947300d0b']:
                files = os.listdir(middle_file_dir)
                for file in files:
                    if file.startswith(subject) and file.endswith('.json'):
                        file_path = os.path.join(middle_file_dir, file)
                        print(f"middle file path: {file_path}")
                        tmp_wave = load_wave_middleSchool(file_path)
                        subject_wave += tmp_wave

            elif subject in (middle_subjects_old_neg + middle_subjects_old_pos):
                files = os.listdir(middle_file_old_dir)
                for file in files:
                    if file.startswith(subject) and file.endswith('.json'):
                        file_path = os.path.join(middle_file_old_dir, file)
                        print(f"middle file path old: {file_path}")
                        tmp_wave = load_wave_middleSchool_old(file_path)
                        subject_wave += tmp_wave
            
            if len(subject_wave) < (sample_freq * (sample_window_minute) * 60):
                print(f"Small time for subj: {subject} is: {len(subject_wave)/(sample_freq*60)}")
                ignored_id.append({'id': id, 'Time Length of data in minute ': f"{len(subject_wave)/(125 * 60):.02f}"})
                continue
            else:
                print(f"Recording time for subj: {subject} is: {len(subject_wave)/(sample_freq*60)}")
                subject_dataset = generate_train_feature(subject, subject_wave, label, profile_dict[subject][0], profile_dict[subject][1], sample_freq, sample_window_minute, data_aug_times, if_gender = True)
                dataset = pd.concat([dataset, subject_dataset], ignore_index=True)
    return dataset




In [31]:
pos_dataset = generate_dataset(middle_subjects_old_pos, label=1, data_aug_times=4)
neg_dataset = generate_dataset(all_subjects_neg, label=0, data_aug_times=1)

middle file path old: C:\Users\Administrator\Desktop\PPG_dataset\PPG_Dataset_ALL\middle_school\ppg_data\138681313570679646396533197328690174535.json
Recording time for subj: 138681313570679646396533197328690174535 is: 2.95
middle file path old: C:\Users\Administrator\Desktop\PPG_dataset\PPG_Dataset_ALL\middle_school\ppg_data\82498448730405369670702745834853438447.json
Recording time for subj: 82498448730405369670702745834853438447 is: 2.966666666666667
middle file path old: C:\Users\Administrator\Desktop\PPG_dataset\PPG_Dataset_ALL\middle_school\ppg_data\70055199254646997137347024168883235522.json
Recording time for subj: 70055199254646997137347024168883235522 is: 2.9833333333333334
middle file path old: C:\Users\Administrator\Desktop\PPG_dataset\PPG_Dataset_ALL\middle_school\ppg_data\277473614294071127937667446265054569611.json
Recording time for subj: 277473614294071127937667446265054569611 is: 3.0
middle file path old: C:\Users\Administrator\Desktop\PPG_dataset\PPG_Dataset_ALL\middl

In [32]:
columns_drop = []
for col in pos_dataset.columns[:-4]:
    nan_count = (pos_dataset[col] == 0).sum() 
    if nan_count > (len(pos_dataset) // 4): 
        columns_drop.append(col)


pos_dataset_final = pos_dataset.drop(columns=columns_drop, inplace=False)
neg_dataset_final = neg_dataset.drop(columns=columns_drop, inplace=False)

In [33]:
print(f"After remove the invalid columns, train pos shape: {pos_dataset_final.shape}")
print(f"After remove the invalid columns, train neg shape: {neg_dataset_final.shape}")

After remove the invalid columns, train pos shape: (430, 32)
After remove the invalid columns, train neg shape: (2171, 32)


In [34]:
profile_columns = ['wave_quality', 'age', 'gender', 'label', 'id']

In [35]:
# 合并训练集
train_combined = pd.concat([pos_dataset_final, neg_dataset_final])
# 需要归一化的列
columns_to_normalize = train_combined.columns.difference(profile_columns)
current_date = datetime.datetime.now().strftime("%Y-%m-%d")
outdir = os.path.join(os.getcwd(), 'outdir')
# 从CSV文件中读取最小值和最大值
min_max_df = pd.read_csv(f'{outdir}/ppg_depression_norm_minmax_values_{current_date}.csv', index_col=0)
min_values = min_max_df['min']
max_values = min_max_df['max']

# 定义min-max归一化函数
def normalize(df, min_values, max_values):
    return (df - min_values) / (max_values - min_values)


# 对测试集进行归一化
pos_dataset_final[columns_to_normalize] = normalize(pos_dataset_final[columns_to_normalize], min_values, max_values)
neg_dataset_final[columns_to_normalize] = normalize(neg_dataset_final[columns_to_normalize], min_values, max_values)


In [36]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [37]:
sample_folder = os.path.join(os.getcwd(), 'samples')
os.makedirs(sample_folder, exist_ok=True)

In [38]:
test_pos_path = os.path.join(sample_folder, 'testing_positive.pkl')
test_neg_path = os.path.join(sample_folder, 'testing_negative.pkl')

In [39]:
pos_dataset_final.to_pickle(test_pos_path)
neg_dataset_final.to_pickle(test_neg_path)